# SOG

Weighted Sum of Gaussians objectve function:

\begin{equation}
\label{eq:funct1}
f(x_n^{(k)})= -\sum_{p=1}^{P} c_p\exp \Bigg\{ {-\frac{1}{2 \sigma^2}(x_n^{(k)}-x_{0p})^T A_p^T \Sigma_p A_p(x_n^{(k)}-x_{0p})}\Bigg\}\, .
\end{equation}
where $x_n^{(k)}$ is the $n$-th point after $k$ iterations of anti-gradient descent, $P$ is the number of Gaussian densities; $A_p$ is a random rotation matrix of size $d\times d$; $\Sigma_p$ is a diagonal positive definite matrix of size $d\times d$ with smallest and largest eigenvalues $\lambda_{min}$ and $\lambda_{max}$ respectively;  $x_{0p} \in \mathfrak{X}=[0,1]^d$ (centers of the Gaussian densities); $c_p$ is a fixed constant and $p=1,...,P$.

Note that anti-gradient descent iterations are terminated at the smallest $k=K_n$ such that $\nabla f(x_n^{(k)}) < \delta$, where $\delta$ is some small positive constant. 



# Import libraries

In [1]:
import numpy as np 
from numpy import linalg as LA
import pandas as pd
import matplotlib.pyplot as plt; plt.rcdefaults()
import textwrap

import metod_alg as mt
from metod_alg import metod_algorithm_functions as mt_alg
from metod_alg import objective_functions as mt_obj
import tqdm
import seaborn as sns
import SALib
from SALib.sample import sobol_sequence

# Results

For all results $d=20$ and $N=100$.

In [16]:
d = 20
p = 10
sig_sq = 0.7
beta_list = [0.1]
m_list = [1]

avg_grad = np.zeros((len(m_list) * len(beta_list), 100, 500))

total_no_local_minimizers_metod = np.zeros((len(m_list), len(beta_list), 100))
time_taken_metod = np.zeros((len(m_list), len(beta_list), 100))
extra_descents_metod = np.zeros((len(m_list), len(beta_list), 100))
func_val_metod = np.zeros((len(m_list), len(beta_list), 100))

df_mult = pd.read_csv('sog_sd_metod_beta_0.1_m=1_d=20_p=10_random_sig_%s_100.csv' % (sig_sq))
total_no_local_minimizers_mult = np.array(df_mult['number_minimizers_per_func_multistart'])
time_taken_mult = np.array(df_mult['time_multistart'])
func_val_mult = np.array(df_mult['min_func_val_multistart'])
test = np.array(df_mult['number_minimizers_per_func_metod'])

# index_all = 0
# index_m = 0
# for m in m_list:
#     index_beta = 0
#     for beta in beta_list:
#         df_metod = pd.read_csv('sog_metod_beta_%s_m=%s_d=20_p=10_sig_%s_random_100.csv'% (beta, m, sig_sq))
#         total_no_local_minimizers_metod[index_m, index_beta] = np.array(df_metod['number_minimizers_per_func_metod'])
#         time_taken_metod[index_m, index_beta] = np.array(df_metod['time_metod'])
#         func_val_metod[index_m, index_beta] = np.array(df_metod['min_func_val_metod'])
#         if beta == 0.1 and m == 1:
#             assert(np.all(total_no_local_minimizers_metod[index_m, index_beta] == test))
#         extra_descents_metod[index_m, index_beta] = np.array(df_metod['number_extra_descents_per_func_metod'])

#         avg_grad[index_all] = np.genfromtxt('sog_grad_norm_beta_%s_m=%s_d=20_p=10_random_sig_%s_100.csv'% (beta, m, sig_sq), delimiter=',')
#         index_beta += 1
#         index_all += 1
#     index_m += 1

In [17]:
sig_07_min = total_no_local_minimizers_mult

In [13]:
sig_06_min = total_no_local_minimizers_mult

In [18]:
sig_07_test = test

In [14]:
sig_06_test = test

In [19]:
sig_07_ext = np.array(df_mult['number_extra_descents_per_func_metod'])

In [15]:
sig_06_ext  = np.array(df_mult['number_extra_descents_per_func_metod'])

In [24]:
np.sum(sig_07_min  == sig_07_test)

96

In [25]:
np.sum(sig_06_min  == sig_06_test)

95

In [26]:
np.mean(sig_07_ext)

64.63

In [27]:
np.mean(sig_06_ext)

67.83

In [ ]:
total_no_minimizers_prop = total_no_local_minimizers_metod / total_no_local_minimizers_mult

In [ ]:
time_taken_prop = time_taken_metod / time_taken_mult

In [ ]:
for j in range(len(beta_list) * len(m_list)):
    for k in range(j+1, len(beta_list) * len(m_list)):
            assert(np.all(np.round(avg_grad[k], 5) == np.round(avg_grad[j], 5)))

In [ ]:
same_global_min = np.zeros((len(m_list), len(beta_list)))
same_minimizer = np.zeros((len(m_list), len(beta_list)))
mean_excessive_descents = np.zeros((len(m_list), len(beta_list)))
for i in range(len(m_list)):
    for j in range(len(beta_list)):
        assert(np.all(total_no_local_minimizers_mult >=  total_no_local_minimizers_metod[i, j]))
        same_global_min[i, j] = np.where(np.round(func_val_mult, 2) == np.round(func_val_metod[i, j], 2))[0].shape[0]
        same_minimizer[i, j] = np.sum(total_no_local_minimizers_metod[i, j]) / np.sum(total_no_local_minimizers_mult)
        mean_excessive_descents[i, j] = np.mean(extra_descents_metod[i, j])

In [ ]:
same_global_min

In [ ]:
same_minimizer

In [ ]:
number_its = np.genfromtxt('sog_no_its_mult_beta_0.1_m=1_d=4_p=10_random_sig_%s_500.csv' % (sig_sq), delimiter=',')
store_mean_its = np.zeros((100))
store_min_its = np.zeros((100))
store_mean_norm_grad = np.zeros((100))
for j in range(100):
    store_mean_its[j] = np.mean(number_its[j])
    store_min_its[j] = np.min(number_its[j])
    store_mean_norm_grad[j] = 1 / np.mean(avg_grad[0][j])

In [ ]:
store_mean_its

In [ ]:
store_min_its

In [ ]:
store_mean_norm_grad

In [ ]:
def set_box_color(bp, color):
    """Set colour for boxplot."""
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)
    
def create_boxplots_ratio(arr1, arr2, labels, ticks):
    plt.figure(figsize=(7, 5))
    
    max_num = max(np.max(arr1), np.max(arr2))
    plt.ylim(0, max_num + 0.1)
    bpl = plt.boxplot(arr1.T,
                      positions=np.array(range(len(arr1)))*2.0-0.4)
    bpr = plt.boxplot(arr2.T,
                      positions=np.array(range(len(arr2)))*2.0+0.4)
    set_box_color(bpl, 'green')
    set_box_color(bpr, 'navy')
    plt.plot([], c='green', label=labels[0])
    plt.plot([], c='navy', label=labels[1])
    plt.legend(bbox_to_anchor=(0.99, 1.025), loc='upper left',
               prop={'size': 15})
    plt.xlabel(r'$\beta$', size=14)
    plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks, size=15)
    plt.yticks(fontsize=14)
    plt.tight_layout()

# Total number of local minima

In [ ]:
ticks = [beta_list[0],
          beta_list[1],
          beta_list[2]]
labels = [r'$M =$ %s' % (m_list[0]),
        r'$M =$ %s' % (m_list[1])]

In [ ]:
create_boxplots_ratio(total_no_minimizers_prop[0], total_no_minimizers_prop[1], labels, ticks)

# Efficiency

In [ ]:
create_boxplots_ratio(time_taken_prop[0], time_taken_prop[1], labels, ticks)

# Excessive descents

In [ ]:
create_boxplots_ratio(extra_descents_metod[0], extra_descents_metod[1], labels, ticks)